# Topic modeling

Topic modeling is a type of statistical modeling for discovering the abstract “topics” that occur in a collection of documents. Latent Dirichlet Allocation (LDA) is an example of topic model and is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
from collections import Counter
import operator
import numpy as np
import nltk
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')

In [50]:
search_news_pro=pd.read_csv('/work/news-topic-modeling/data/processed/processed_search_news_dataset.csv')

In [51]:
latest_news_pro=pd.read_csv('/work/news-topic-modeling/data/processed/processed_latest_news_dataset.csv')

In [52]:
search_news_pro

,Unnamed: 0.1,Unnamed: 0,author,title,description,url,urlToImage,Date,content,source.id,Source
0,0,0,Sarah Fielding,Bing Search is coming to ChatGPT,ChatGPT no longer needs to rely on its own mod...,https://www.engadget.com/bing-search-is-coming...,https://s.yimg.com/uu/api/res/1.2/zfz5CvzbQGa_...,2023-05-23 15:00:01,ChatGPT no longer needs to rely on its own mod...,engadget,Engadget
1,1,1,James Vincent,Apple restricts employees from using ChatGPT o...,Apple says employees shouldn’t use generative ...,https://www.theverge.com/2023/5/19/23729619/ap...,https://cdn.vox-cdn.com/thumbor/pySNd0fs2M5PoH...,2023-05-19 08:29:26,Apple has restricted employees from using AI t...,the-verge,The Verge
2,2,2,Lauren Goode,ChatGPT Now Has an iPhone App,Six months after OpenAI’s silver-tongued chatb...,https://www.wired.com/story/chatgpt-iphone-app/,https://media.wired.com/photos/646583619d819fb...,2023-05-18 16:57:28,If you’ve searched for “ChatGPT” in Apple’s Ap...,wired,Wired
3,3,3,David Nield,Supercharge Your ChatGPT Prompts With Auto-GPT,The new tool basically transforms ChatGPT into...,https://www.wired.com/story/chatgpt-prompts-au...,https://media.wired.com/photos/6468145c0124a91...,2023-05-21 11:00:00,The capabilities of AI tools are progressing r...,wired,Wired
4,4,4,"Amanda Hoover, Samantha Spengler","For Some Autistic People, ChatGPT Is a Lifeline",The chatbot can help rehearse communication sk...,https://www.wired.com/story/for-some-autistic-...,https://media.wired.com/photos/647146ed0cbc159...,2023-05-30 11:00:00,Autism affects people in many different ways a...,wired,Wired
...,...,...,...,...,...,...,...,...,...,...,...
95,95,95,Maya Posch,ChatGPT v. The Legal System: Why Trusting Chat...,"Recently, an amusing anecdote made the news he...",https://hackaday.com/2023/05/29/chatgpt-v-the-...,https://hackaday.com/wp-content/uploads/2023/0...,2023-05-29 14:00:44,"Recently, an amusing anecdote made the news he...",NaN,Hackaday
96,96,96,Jay Peters,Elon Musk’s lawyer accuses Microsoft of abusin...,Elon Musk’s personal lawyer Alex Spiro sent a ...,https://www.theverge.com/2023/5/18/23729216/tw...,https://cdn.vox-cdn.com/thumbor/l0_-U17tDi8ziX...,2023-05-18 22:44:05,Elon Musk’s personal lawyer Alex Spiro sent a ...,the-verge,The Verge
97,97,97,Sindhu Sundar,ChatGPT copycats are taking over the App Store...,ChatGPT debuted on the App Store earlier this ...,https://www.businessinsider.com/openai-chatgpt...,https://i.insider.com/646b800663c86a0018cd1c10...,2023-05-22 18:24:07,Several apps that seem to rip off OpenAI's Cha...,business-insider,Business Insider
98,98,98,msmash,China Reports First Arrest Over Fake News Gene...,A man in China's Gansu province has been detai...,https://news.slashdot.org/story/23/05/10/16472...,https://a.fsdn.com/sd/topics/china_64.png,2023-05-10 16:47:00,A man in China's Gansu province has been detai...,NaN,Slashdot.org


In [53]:
latest_news_pro

,Unnamed: 0.1,Unnamed: 0,author,title,description,url,urlToImage,Date,content,source.id,Source
0,0,0,NaN,At least 261 dead in India's worst train accid...,At least 261 people have died in India's worst...,https://www.reuters.com/world/india/indian-tra...,https://www.reuters.com/resizer/CmuQ9DbJWBJVvZ...,2023-06-03 13:52:00,"\n\n\n\n\n\n\n\n\n\n\n\n\n\nBAHANAGA, India, J...",reuters,Reuters
1,1,1,NaN,"Three Israeli soldiers, Egyptian officer kille...",Three Israeli soldiers and an Egyptian securit...,https://www.reuters.com/world/middle-east/two-...,https://www.reuters.com/resizer/ISSoZgnJbKeO8k...,2023-06-03 13:48:00,"\n\n\n\n\n\n\n\n\n\n\n\n\n\nJERUSALEM/CAIRO, J...",reuters,Reuters
2,2,2,NaN,European Space Agency Releases New Mosaic Of M...,The mosaic provides fascinating information ab...,https://www.ndtv.com/science/european-space-ag...,https://c.ndtvimg.com/2023-06/0bke9tqg_new-mos...,2023-06-03 13:10:43,The mosaic was created using data from Mars Ex...,NaN,NDTV News
3,3,3,Al Jazeera,Three Israeli soldiers killed in shootings nea...,Israel army says attack by man dressed in an E...,https://www.aljazeera.com/news/2023/6/3/israel...,https://www.aljazeera.com/wp-content/uploads/2...,2023-06-03 12:56:15,Attack by a man dressed in an Egyptian securit...,al-jazeera-english,Al Jazeera English
4,4,4,MD Ijaj Khan,Kiljoy! No more 'Hey Siri'! Apple set to make ...,Apple may eliminate the 'Hey Siri' trigger phr...,https://tech.hindustantimes.com/tech/news/kilj...,https://images.hindustantimes.com/tech/img/202...,2023-06-03 12:53:01,The phrase 'Hey Siri' has become an iconic fea...,NaN,Hindustan Times
5,5,5,The Guardian US,Ron DeSantis thanks then attacks heckler who c...,NaN,https://news.google.com/rss/articles/CBMiTGh0d...,NaN,2023-06-03 12:38:00,A South Carolina heckler called Ron DeSantis a...,google-news,Google News
6,6,6,ET HealthWorld,Study reveals possible target for treating neu...,Researchers at the University of Rochester's D...,https://health.economictimes.indiatimes.com/ne...,https://etimg.etb2bimg.com/thumb/msid-10072815...,2023-06-03 12:08:45,Advt\n\nJoin the community of 2M+ industry pro...,the-times-of-india,The Times of India
7,7,7,NaN,3 Teens Arrested For Killing And Eating New Yo...,The teens sneaked into the pond area in the mi...,https://www.ndtv.com/world-news/3-teens-arrest...,https://c.ndtvimg.com/2023-06/ikac0ep_swan-_62...,2023-06-03 12:04:06,"The female swan, named Faye, was a resident of...",NaN,NDTV News
8,8,8,Shailaja Mohapatra,ITC and TCS among top dividend-paying stocks i...,Most retail investors when buying stocks focus...,https://www.moneycontrol.com/news/business/mar...,https://images.moneycontrol.com/static-mcnews/...,2023-06-03 12:00:15,Representative Image\n\nlive bse live\n\nnse l...,NaN,Moneycontrol
9,9,9,CNET,Watch FA Cup Final Soccer: Livestream Man City...,NaN,https://news.google.com/rss/articles/CBMiemh0d...,NaN,2023-06-03 12:00:00,Bitter local rivals Manchester City and Manche...,google-news,Google News


### Clean the Data

In [54]:
latest_news_pro.fillna('', inplace=True)

In [55]:
search_news_pro.fillna('', inplace=True)

In [56]:
latest_news_pro['content']=latest_news_pro['content'].str.replace("[^a-zA-Z#]", " ")

In [57]:
search_news_pro['content']=search_news_pro['content'].str.replace("[^a-zA-Z#]", " ")

In [58]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [59]:
import re

In [60]:
def clean_text(text):
    #for token in text:
    text=re.sub('[^A-Za-z0-9]+', ' ', text).strip()
    text =  ' '.join([w.lower() for w in text.split() if len(w)>3])
    return text


In [61]:
search_news_pro['content'] = search_news_pro['content'].apply(clean_text)
search_news_pro['content'] = search_news_pro['content'].apply(clean_doc)
search_news_pro['content'][0]

'chatgpt longer needs rely models plugins provide information users microsoft announced bing search integrating into chatgpt provide more relevant potentially newer responses bing will default search experience allowing chatgpt pull answers from internet provide citations microsoft openai have worked together closely over last months with bing itself running with some tailoring searches bing works displaying responses detailed summary form sources facts suggestions with footnotes sorts features bring chatgpt possibly provide more precise information previously chatgpt relied solely individual plugins access recent information other bing updates plugin centric including single platform developers create submit them chatgpt bing microsoft copilot more bing adding slew companies chat plugin options such expedia zillow klarna targeted communication topics like travel shopping common plugin platform also being natively integrated into microsoft edge where bing already built exact timeline e

In [62]:
def stopWordRemoval(text):
    text = word_tokenize(text)
    text = ' '.join([word for word in text if word not in custom])
    return text

In [63]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [64]:
stopwords_set = set(stopwords.words('english'))
custom = list(stopwords_set)+list(punctuation)
stop = stopwords_set
exclude = set(punctuation)
lemma = WordNetLemmatizer()


In [65]:
search_news_pro['content'] = search_news_pro['content'].apply(stopWordRemoval)
search_news_pro['content']

0     chatgpt longer needs rely models plugins provi...
1     apple restricted employees using tools like op...
2     searched chatgpt apple store since chatbot lau...
3     capabilities tools progressing rapidly google ...
4     autism affects people many different ways indi...
                            ...                        
95    recently amusing anecdote made news headlines ...
96    elon musk personal lawyer alex spiro sent lett...
97    several apps seem openai chatgpt populating ap...
98    china gansu province detained allegedly using ...
99    biden administration officials divided aggress...
Name: content, Length: 100, dtype: object

In [66]:
def lemData(text):
    text = word_tokenize(text)
    newText = []
    for word in text:
        newText.append(lemmatizer.lemmatize(word))
    return ' '.join(newText)

In [67]:
search_news_pro['content'] = search_news_pro['content'].apply(lemData)
search_news_pro['content']

0     chatgpt longer need rely model plugins provide...
1     apple restricted employee using tool like open...
2     searched chatgpt apple store since chatbot lau...
3     capability tool progressing rapidly google mic...
4     autism affect people many different way indivi...
                            ...                        
95    recently amusing anecdote made news headline p...
96    elon musk personal lawyer alex spiro sent lett...
97    several apps seem openai chatgpt populating ap...
98    china gansu province detained allegedly using ...
99    biden administration official divided aggressi...
Name: content, Length: 100, dtype: object

In [68]:
# Function to lemmatize and remove the stopwords
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = " ".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())
    return normalized

In [70]:
#search_news_pro['content'] = search_news_pro['content'].apply(clean)
search_news_pro['content'][10]

'opera testing sidebar browser feature called aria powered openai chatgpt generate text write code answer question according blog post opera website aria life within opera sidebar left side screen clicking icon reveals panel interact unlike standard version chatgpt chatbot opera browser surface date information across also capable answering question opera company say knowledgeable browser whole database support documentation image opera introduction aria build upon feature opera already february opera added chatgpt sidebar allowing generate summary webpage article visiting browser also rolled prompt feature offer shortcut various tool change depending page text highlight let thing like summarize text create tweet based reading explanation specific concept'

In [ ]:
def clean_text(text):
    #for token in text:
    text=re.sub('[^A-Za-z0-9]+', ' ', text).strip()
    text=re.sub(r'[0-9]+', '', text)
    return text

In [ ]:
latest_news_pro['content'].apply(clean_text)

In [ ]:
tokenized_latest_news_pro.apply(f1)

In [13]:
def clean_doc(text):        
    #for token in text:
    text =  ' '.join([w.lower() for w in text.split() if len(w)>3])
    return text

def stopWordRemoval(text):
    text = word_tokenize(text)
    text = ' '.join([word for word in text if word not in custom])
    return text
# Initializing wordnet lemmatizer
lemmatizer = WordNetLemmatizer()

def lemData(text):
    text = word_tokenize(text)
    newText = []
    for word in text:
        newText.append(lemmatizer.lemmatize(word))
    return ' '.join(newText)

stopwords_set = set(stopwords.words('english'))
custom = list(stopwords_set)+list(punctuation)
stop = stopwords_set
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

# Function to lemmatize and remove the stopwords
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = " ".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized


def topic_model():
    search_news_pro=pd.read_csv(str(Path(__file__).resolve().parent.parent.parent)+'\\data\\processed\\processed_search_news_dataset.csv')
    latest_news_pro=pd.read_csv(str(Path(__file__).resolve().parent.parent.parent)+'\\data\\processed\\processed_latest_news_dataset.csv')

    latest_news_pro.fillna('', inplace=True)
    search_news_pro.fillna('', inplace=True)




    tokenized_search_news_pro = search_news_pro['content'].apply(clean_doc)
    tokenized_latest_news_pro = latest_news_pro['content'].apply(clean_doc)


    tokenized_latest_news_pro = tokenized_latest_news_pro.apply(stopWordRemoval)
    tokenized_search_news_pro = tokenized_search_news_pro.apply(stopWordRemoval)

    tokenized_search_news_pro = tokenized_search_news_pro.apply(lemData)
    tokenized_latest_news_pro = tokenized_latest_news_pro.apply(lemData)

    list_search_news_pro = tokenized_search_news_pro.tolist()
    list_latest_news_pro = tokenized_latest_news_pro.tolist()

    clean_search_news_pro = [clean(doc).split() for doc in list_search_news_pro]
    clean_latest_news_pro = [clean(doc).split() for doc in list_latest_news_pro]


    #LDA

    # Creating the dictionary 
    dictionary1 = corpora.Dictionary(clean_search_news_pro)
    # Creating the corpus
    search_news_term_matrix = [dictionary1.doc2bow(doc) for doc in clean_search_news_pro]

    # Creating the dictionary 
    dictionary2 = corpora.Dictionary(clean_latest_news_pro)
    # Creating the corpus
    latest_news_term_matrix = [dictionary2.doc2bow(doc) for doc in clean_latest_news_pro]

    # Creating the LDA model
    search_ldamodel = LdaModel(corpus=search_news_term_matrix, num_topics=4,id2word=dictionary1, random_state=20, passes=30)
    latest_ldamodel = LdaModel(corpus=latest_news_term_matrix, num_topics=4,id2word=dictionary2, random_state=20, passes=30)

    #saving models to disk.

    temp_file1 = datapath(str(Path(__file__).resolve().parent.parent.parent)+'\\models\\search_lda_model')
    search_ldamodel.save(temp_file1)
    temp_file2 = datapath(str(Path(__file__).resolve().parent.parent.parent)+'\\models\\latest_lda_model')
    latest_ldamodel.save(temp_file2)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### Handling the Stop-words

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stopwords_set = set(stopwords.words('english'))
custom = list(stopwords_set)+list(punctuation)

In [ ]:
def stopWordRemoval(text):
    text = word_tokenize(text)
    text = ' '.join([word for word in text if word not in custom])
    return text

In [ ]:
tokenized_latest_news_pro = tokenized_latest_news_pro.apply(stopWordRemoval)
tokenized_latest_news_pro

In [ ]:
tokenized_search_news_pro = tokenized_search_news_pro.apply(stopWordRemoval)
tokenized_search_news_pro

### Lemmatization

In [ ]:
# Initializing wordnet lemmatizer
lemmatizer = WordNetLemmatizer()

def lemData(text):
    text = word_tokenize(text)
    newText = []
    for word in text:
        newText.append(lemmatizer.lemmatize(word))
    return ' '.join(newText)



In [ ]:
tokenized_search_news_pro = tokenized_search_news_pro.apply(lemData)
tokenized_search_news_pro

In [ ]:
tokenized_latest_news_pro = tokenized_latest_news_pro.apply(lemData)
tokenized_latest_news_pro

### TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialising the tfidf vectorizer with the default stopword list 
tfidf = TfidfVectorizer(stop_words="english", max_features= 1000, max_df = 0.5, smooth_idf=True)

In [ ]:
vector =tfidf.fit_transform(tokenized_latest_news_pro)

#Converting vector into an array
X= vector.toarray()
pd.DataFrame(X)

In [ ]:
vector =tfidf.fit_transform(tokenized_search_news_pro)

#Converting vector into an array
X= vector.toarray()
pd.DataFrame(X)

### Gensim LSI & LDA

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim.models.lsimodel import LsiModel
from gensim import corpora
from pprint import pprint

In [ ]:
stop = stopwords_set
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

# Function to lemmatize and remove the stopwords
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized



In [ ]:
# Creating a list of documents from the complaints column
list_search_news_pro = tokenized_search_news_pro.tolist()

# Implementing the function for all the complaints of list_of_docs
clean_search_news_pro = [clean(doc).split() for doc in list_search_news_pro]
print(clean_search_news_pro[:2])

In [ ]:
# Creating a list of documents from the complaints column
list_latest_news_pro = tokenized_latest_news_pro.tolist()

# Implementing the function for all the complaints of list_of_docs
clean_latest_news_pro = [clean(doc).split() for doc in list_latest_news_pro]
print(clean_latest_news_pro[:2])

In [ ]:
clean_latest_news_pro[7]

In [ ]:
string = "|Special $#! “characters -:@#$&=^*!?~-   /spaces 888323"
' '.join(e for e in string.split(' ') if e.isnum())

In [ ]:
import re

In [ ]:
re.sub('[^A-Za-z0-9]+', ' ', string).strip()

In [ ]:
latest_news_raw['content']
  #=latest_news_raw['content'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
def f(t):
    

### LSI

In [ ]:
# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary1 = corpora.Dictionary(clean_latest_news_pro)

# Creating the corpus
latest_news_term_matrix = [dictionary1.doc2bow(doc) for doc in clean_latest_news_pro]

# Creating the LSi model
lsimodel = LsiModel(corpus=latest_news_term_matrix, num_topics=5, id2word=dictionary1)
pprint(lsimodel.print_topics())

In [ ]:
# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary2 = corpora.Dictionary(clean_search_news_pro)

# Creating the corpus
search_news_term_matrix = [dictionary2.doc2bow(doc) for doc in clean_search_news_pro]

# Creating the LSi model
lsimodel = LsiModel(corpus=search_news_term_matrix, num_topics=5, id2word=dictionary2)
pprint(lsimodel.print_topics())

### LDA

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]


In [ ]:
import string
from gensim.models.ldamodel import LdaModel

# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary3 = corpora.Dictionary(clean_search_news_pro)

# Creating the corpus
search_news_term_matrix = [dictionary3.doc2bow(doc) for doc in clean_search_news_pro]

# Creating the LDA model
ldamodel = LdaModel(corpus=search_news_term_matrix, num_topics=4,id2word=dictionary3, random_state=20, passes=30)

# printing the topics
pprint(ldamodel.print_topics())

In [ ]:
ldamodel[search_news_term_matrix]

In [ ]:
import string
from gensim.models.ldamodel import LdaModel

# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary4 = corpora.Dictionary(clean_latest_news_pro)

# Creating the corpus
latest_news_term_matrix = [dictionary4.doc2bow(doc) for doc in clean_latest_news_pro]

# Creating the LDA model
ldamodel = LdaModel(corpus=latest_news_term_matrix, num_topics=5,id2word=dictionary4, random_state=20, passes=30)

# printing the topics
pprint(ldamodel.print_topics())

### test

In [ ]:
def format_topics_sentences(ldamodel=ldamodel, corpus=search_news_term_matrix, texts=clean_search_news_pro):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=search_news_term_matrix, texts=clean_search_news_pro)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(15)

In [ ]:

# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
import plotly.express as px
import pandas as pd
from collections import Counter
from plotly.subplots import make_subplots

topics = ldamodel.show_topics(formatted=False)
data_flat = [w for w_list in clean_search_news_pro for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i, weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])

# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=("Topic: 0", "Topic: 1", "Topic: 2", "Topic: 3"),
                  shared_yaxes=True, vertical_spacing=0.1)

# Configure colors
colors = [color[4:] for color in mcolors.TABLEAU_COLORS]


# Iterate through each subplot
for i, subplot in enumerate(fig['layout']['annotations']):
    topic_id = i
    topic_df = df.loc[df.topic_id == topic_id, :]

    # Add Word Count Bar Trace
    fig.add_trace(
        go.Bar(
            x=topic_df['word'],
            y=topic_df['word_count'],
            marker=dict(color=colors[i], opacity=0.3),
            width=0.5,
            name='Word Count',
            showlegend=False
        ),
        row=(i // 2) + 1, col=(i % 2) + 1
    )

    # Add Importance Bar Trace
    fig.add_trace(
        go.Bar(
            x=topic_df['word'],
            y=topic_df['importance'],
            marker=dict(color=colors[i]),
            width=0.2,
            name='Weights',
            showlegend=False
        ),
        row=(i // 2) + 1, col=(i % 2) + 1
    )

    # Set y-axis labels and ranges
    fig.update_yaxes(title_text='Word Count', row=(i // 2) + 1, col=(i % 2) + 1, range=[0, 3500])
    fig.update_yaxes(title_text='Weights', row=(i // 2) + 1, col=(i % 2) + 1, secondary_y=True, range=[0, 0.03])

    # Rotate x-axis labels
    fig.update_xaxes(tickangle=30, row=(i // 2) + 1, col=(i % 2) + 1)

# Set layout title
fig.update_layout(title_text='Word Count and Importance of Topic Keywords', title_font=dict(size=22))

# Update subplot axes layout
fig.update_layout(
    height=600,
    width=1200,
    showlegend=False
)

# Show the plot
fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from collections import Counter

topics = ldamodel.show_topics(formatted=False)
data_flat = [w for w_list in clean_search_news_pro for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i, weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])

# Create figure
fig = go.Figure()

cols = [color for color in pio.templates['simple_white'].layout.colorway]

topic_id = 2  # Select the topic ID you want to plot

topic_df = df.loc[df.topic_id == topic_id, :]

# Add Word Count Bar Trace
fig.add_trace(
    go.Bar(
        
        x=topic_df['word'],
        y=topic_df['word_count'],
        marker=dict(color=cols[topic_id], opacity=0.3),
        width=0.5,
        name='Word Count',
        yaxis='y1'
    )
)

# Add Importance Bar Trace
fig.add_trace(
    go.Bar(
        x=topic_df['word'],
        y=topic_df['importance'],
        marker=dict(color=cols[topic_id]),
        width=0.2,
        name='Weights',
        yaxis='y2'
    )
)

fig.update_layout(
    xaxis=dict(tickangle=30),
    yaxis=dict(title='Word Count', color=cols[topic_id], range=[0, 3500]),
    yaxis2=dict(title='Weights', color=cols[topic_id], range=[0, 0.03], overlaying='y', side='right'),
    barmode='overlay',
    legend=dict(x=0, y=1.2, orientation='h')
)

fig.update_traces(showlegend=True, selector=dict(type='bar'))
fig.update_layout(template='plotly_dark')

fig.update_layout(
    title='Word Count and Importance of Topic Keywords (Topic {})'.format(topic_id),
    title_font=dict(size=22),
    height=600,
    width=800,
    showlegend=True
)

fig.show()


### Evaluation Metric

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Perplexity
perplexity_lda = ldamodel.log_perplexity(latest_news_term_matrix)
print('\nPerplexity: ', perplexity_lda)  


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=clean_latest_news_pro, dictionary=dictionary4, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### pyLDAvis Visualization

In [ ]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, latest_news_term_matrix, dictionary4)
vis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, search_news_term_matrix, dictionary3)
vis

In [ ]:
pyLDAvis.save_html(vis, '/work/news-topic-modeling/reports/lda.html')

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stopwords,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = ldamodel.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    print(topic_words)
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
#fig.savefig('word_cloud.png')
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e25ccbc-e36d-4741-b648-71cf16013ae6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>